# Project 3: Predicting Winners in Soccer Matches

Predicting the outcome of a sporting event is a challenging task, so much so that an entire industry exists to relieve  those who believe they can do so of their money. Despite the challenging nature of the task, there are a few people who manage to make their living (and a very good living at that!) making predictions and betting on the outcomes. Your job is to try your hand at this with a basic model.

## The Data

For this project, you are going to use a large *relational database* of publicly available information on European soccer matches that is available [here](https://www.kaggle.com/hugomathien/soccer/kernels). This database is a SQLite database. A relational database is a data structure which contains several related tables, where each table is similar to a dataframe. There are a couple of advantages to having the data in a database: for one thing, you can 'peek' at the data without loading it all into memory. Also, databases tend to be strongly typed, so that you always know the type of data that is loaded in each column (though actually SQLite is much looser in this regard than other database software). 

## The Objective(s)

Below, you will tag every match with a 1 for home team win or draw and a 0 for away team win. Your goal is to build a model to predict whether the game is a home team win or draw or an away team win. You must:

1. Explore the dataset. What appears to correlate with home team wins/losses? What is missing?
2. Separate the data into a set for training (building the model) and testing. The test set size should not be more than 20% of the full dataset.
3. Build a model to predict who wins. To determine the quality of the model, assess the model by calculating the accuracy. You can write a function to do this or use scikit-learn's `accuracy_score`, which is in the `metrics` submodule.
4. Build a couple of different models (creating a different model means using a different set of predictor variables). Which performs the best on the held-out testing dataset? Bonus points for creativity and thoroughness! You must create and test a minimum of three different models for full credit.

## Teams

1. Evan Royer, Briana Garcia (Durham)
2. Evan O'Reilly, Jacob Mechaber (Durham)
3. Patrick Donovan, Kelvin Nguyen, Robert Patenaude
4. Aaron Broady, Andrew Meyer, Wesley Couturier, Ryan Kiessling
5. Angus Jameson, Danielle LeBoeuf, Craig Vignault 
6. Brent Groden, Stephen McCarter, Aurora VanDeWater

Consider using Git/Github to share and manage your work!

## Due Date

Due by Tuesday, May 2, 3:39 PM. Submit as an HTML file on myCourses.

## Final Thoughts

This is an extremely challenging prediction task. To get a sense of how hard it is, start by making a couple absolutely dumb baseline models: predicting always home team wins, for instance, or predicting wins entirely at random. Measure the accuracy of these models. As you develop more sophisticated models, refer back to these dumb ones. Did your model do better?

This database has been explored quite a bit on [kaggle.com](https://www.kaggle.com). Feel free to use what you see there.

In [1]:
'''
DO NOT CHANGE ANYTHING IN THIS CELL
YOU MUST RUN THIS CELL TO CONTINUE
'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

%matplotlib inline
plt.style.use('bmh')

## Getting the data out of the database

To get data from a database, you must first connect to it from Python. Once you've connected, you then need to create a cursor. A cursor is a temporary memory space where operations can be performed.

In [2]:
'''
CHANGE YOUR PATH IN THE LINE BELOW
'''

path_to_my_database = '../beautiful-game/database.sqlite'
conn = sqlite3.connect(path_to_my_database)  # connect to database
cur = conn.cursor()  # create cursor

## Getting A View of the Database

One way to get an overview of the entire database is to look at the code that created it. This is SQL (Structured Query Language), which you do not know yet, but which is fairly intuitive. We'll execute a command to view the contents of the sqlite_master table, which contains the code that created the underlying tables. Syntax is as follows: first execute, then fetch the results (if you don't fetch them, they exist, but they remain in the cursor's temporary workspace).

In [3]:
'''
DO NOT CHANGE ANYTHING IN THIS CELL
'''

cur.execute("select * from sqlite_master where type == 'table';").fetchall()

[('table',
  'sqlite_sequence',
  'sqlite_sequence',
  4,
  'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table',
  'Player_Attributes',
  'Player_Attributes',
  11,
  'CREATE TABLE "Player_Attributes" (\n\t`id`\tINTEGER PRIMARY KEY AUTOINCREMENT,\n\t`player_fifa_api_id`\tINTEGER,\n\t`player_api_id`\tINTEGER,\n\t`date`\tTEXT,\n\t`overall_rating`\tINTEGER,\n\t`potential`\tINTEGER,\n\t`preferred_foot`\tTEXT,\n\t`attacking_work_rate`\tTEXT,\n\t`defensive_work_rate`\tTEXT,\n\t`crossing`\tINTEGER,\n\t`finishing`\tINTEGER,\n\t`heading_accuracy`\tINTEGER,\n\t`short_passing`\tINTEGER,\n\t`volleys`\tINTEGER,\n\t`dribbling`\tINTEGER,\n\t`curve`\tINTEGER,\n\t`free_kick_accuracy`\tINTEGER,\n\t`long_passing`\tINTEGER,\n\t`ball_control`\tINTEGER,\n\t`acceleration`\tINTEGER,\n\t`sprint_speed`\tINTEGER,\n\t`agility`\tINTEGER,\n\t`reactions`\tINTEGER,\n\t`balance`\tINTEGER,\n\t`shot_power`\tINTEGER,\n\t`jumping`\tINTEGER,\n\t`stamina`\tINTEGER,\n\t`strength`\tINTEGER,\n\t`long_shots`\tINTEGER,\n\t`aggr

## Getting Data From SQLite into Python

As you can see, there are tables like Player_Attributes, Player, Matches, Team_Attributes, and so on. It's easy to bring one of these tables into Pandas with a simple SQL query. In SQL, * is a stand-in for 'everything'. To read a SQL query, keep in mind that you need a connection open, and you need to specify the connection when you write the query.

In [4]:
'''
DO NOT CHANGE ANYTHING IN THIS CELL
'''

players = pd.read_sql_query('select * from Player', conn)  # don't forget to specify the connection
players.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [5]:
matches = pd.read_sql_query('select * from Match', conn)  # specify the connection
print(matches.shape)
matches.head()

(25979, 115)


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


## Complex Queries

You can bring in selected subsets of the database using more complex SQL queries. If you know SQL already, this may be useful. We won't have time to go into complex SQL queries, but it's worth your while to read a little bit about the language; after all, SQL is one of the top languages for data science in its own right. See [this non-python specific SQLite tutorial](https://www.tutorialspoint.com/sqlite/) for an introduction. Fortunately, this dataset is small enough that you should be able to bring most/all of it into memory and thereby work almost entirely in Python.

## Labeling The Matches

The dataset currently doesn't contain a column stating who won and who lost. You'll need to add that. To do so, you can do something like below. You may group draws with home team wins. 

In [6]:
'''
DO NOT CHANGE ANYTHING IN THIS CELL
'''

matches['home_team_win_or_draw'] = 0
matches.loc[matches['home_team_goal'] >= matches['away_team_goal'], 'home_team_win_or_draw'] = 1

# Good Luck!

In [ ]:
# your code here